# Server Module

## 安裝PliotGaea Python Module

From pip install：

> ```shell
> pip install OViewPy
> ```
>

## 初始化Server物件
本章節為PliotGaea Python Module的起始點，在執行所有操作前須先初始化Server物件，提供Module取得Map Server資源的相關連線資訊。

In [ ]:
from OViewPy.server import Server

# 綁定Server物件
server = Server(url="http://127.0.0.1:8080")


綁定Server物件後，即可取得Server相關資訊。

In [ ]:
from OViewPy.server import Server

# 綁定Server物件
server = Server(url="http://127.0.0.1:8080")
# 取得目前Server版本
print("OView Map Server Version：", server.version)
# 取得WMTS URL
print("WMTS URL：", server.wmtsURL)
# 取得WMS URL
print("WMS URL：", server.wmsURL)
# 取得DoCommand URL
print("DoCommand URL：", server.docmdURL)


## 取得圖層列表

綁定Server物件後，可透過`getLayerList`取得2D圖層列表詳細資訊。  
亦可使用`getOViewLayerList`取得3D圖層列表詳細資訊。

In [ ]:
from OViewPy.server import Server

# 綁定Server物件
server = Server(url="http://127.0.0.1:8080")
layerList = server.getLayerList()
OViewLayerList = server.getOViewLayerList()
for list in layerList:
    print(list)
print("===========================================")
for list in OViewLayerList:
    print(list)


## 刪除圖層

如果想要刪除Server中已存在的圖層，可透過`deleteLayer`與`deleteOViewLayer`分別刪除2D及3D圖層。
> 注意！此刪除無法復原，請警慎使用。

In [ ]:
from OViewPy.server import Server

# 綁定Server物件
server = Server(url="http://127.0.0.1:8080")
server.deleteLayer(layerName="ne_10m_lakes")
server.deleteOViewLayer(layerName="ModelSet")


## 上架圖層

### 將圖片上架至伺服器

透過`saveImageToServer`可將圖片(JPG、PNG等)上架至Server

> 需特別注意，上架的圖片需在Server端目錄底下，並有相對應的World File，且確認圖層名稱不是已存在的圖層。

| 參數名稱 | Type | 預設值 | 說明 |
| :-----: | :---: | :---: | :--: |
| imageFilePath | string | None | 圖片檔案位置 |
| layerName | string | None | 上架後的圖層名稱 |
| epsg | int | 4326 | 座標參考系統 |

In [ ]:
from OViewPy.server import Server
from OViewPy.layer import VectorLayer
from OViewPy.varstruct import GeoBoundary
from OViewPy.da import da

# 綁定Server物件
server = Server(url="http://127.0.0.1:8080")
# 綁定Layer物件
layer = VectorLayer(server=server, layerName="Town_MOI")
boundary = GeoBoundary(147522.218692, 2422004.773002,
                       351690.114369, 2813163.248085)
# 取得圖片，取得成功會回傳圖片bytes資料
img = layer.getMapImage(
    boundary=boundary,
    crs="EPSG:3826",
    width=512,
    height=512,
    format="image/png"
)
# 儲存圖片，並生成World File
da.saveImg(
    img=img,
    savePath=".",
    imgName="testImage",
    imgType="png",
    worldFile=True,
    boundary=boundary
)
# 將圖片上架至Server
server.saveImageToServer(
    imageFilePath="D:\\NCHCProject\\jupyter_notebook\\testImage.png",
    layerName="testImage",
    epsg=3826
)


執行結果如下：  

![saveImageToServer](./img/saveImgToServer.png)

## 將Vector File上架至伺服器

透過`saveVectorFileToServer`可將Vector File上架至Server

> 需特別注意，上架的Vector File需在Server端目錄底下，並確認圖層名稱不是已存在的圖層。

| 參數名稱 | Type | 預設值 | 說明 |
| :-----: | :---: | :---: | :--: |
| VectorFilePath | string | None | Vector File位置 |
| layerName | string | None | 上架後的圖層名稱 |
| epsg | int | 4326 | 座標參考系統 |

In [5]:
from OViewPy.server import Server
from OViewPy.layer import VectorLayer
from OViewPy.varstruct import GeoBoundary
from OViewPy.da import da

# 綁定Server物件
server = Server(url="http://127.0.0.1:8080")
# 綁定Layer物件
map = VectorLayer(server=server, layerName="Town_MOI")
# 設定搜尋條件
sql = "County_ID==64"
# 取得向量資料
ret = map.getVectorEntity(epsg=3826, sql=sql)
# 將向量資料存成SHP檔
da.saveAsShapeFile(sourceGeo=ret["geo"],
                   sourceAttr=ret["attr"], fileName="TestShp_Kh2")
# 將SHP檔上架至伺服器
server.saveVectorFileToServer(
    VectorFilePath="D:\\NCHCProject\\jupyter_notebook\\TestShp_Kh2.shp", layerName="TestShp_Kh2", epsg=3826)


Done！


## 將tif檔轉成dem檔，並將dem檔上架伺服器為地形圖層

於3d的轉檔需要較久的時間，因此轉檔的函式會回傳一個id(如: `server.tifToDem`)  
可以透過此id，去跟server詢問進度  
當轉檔成功可以透過`server.save3dToServer`將此圖層上架至伺服器  
`server.save3dToServer`可以透過id或是DB位置將其上架至伺服器

In [ ]:
from OViewPy.server import Server
from OViewPy.varstruct import LAYER_TYPE

# 綁定Server物件
server = Server(url="http://127.0.0.1:8080")
# tif檔轉成dem檔
id = server.tifToDem(inputTifFile="defaultTIF.tif", outputDemFile="defaultDEM2.dem")
# 回傳轉檔進度直到轉檔完成
server.getConvert3dProgressToFinish(id)
# dem檔轉成地形圖層
id = server.convertTerrain("testDem", "testDem.DB", "./defaultDEM2.dem", "4326")
# 將此地形圖層上架至伺服器
server.save3dToServer(dbLayerName="testDem", layerName="testDem", layerType=LAYER_TYPE.OV_TERRAIN.name, dbUrl="./testDem.DB")

## 將傾斜攝影轉檔並上架至伺服器

透過綁定的server物件，並提供欲設定的圖層名稱(`layerName`)及輸出格式(`exportType`)，將傾斜攝影轉檔並上架至伺服器  
由於3d的轉檔需要較久的時間，因此轉檔的函式會回傳一個id(如: `server.convertPhotogrammetry`)  
可以透過此id，去跟server詢問進度  
當轉檔成功透過`server.save3dToServer`依DB位置將圖層上架至伺服器(若輸出檔案格式為3DTILES則不上架)  
| 參數名稱 | Type | 預設值 | 可輸入值 | 說明 |
| :-----: | :---: | :---: | :---: | :--: |
| layerName | string | None | --- | 上架後的圖層名稱 |
| layerDBFile | string | 預設名稱與layerName相同 | --- | 轉檔後產生的DB檔名稱 |
| inputFileName | string | None | --- | 欲轉檔檔案位置 |
| terrainDBFile | string | None | --- | 初始地形圖層位置 |
| terrainName | string | None | --- | 初始地形圖層名稱 |
| exportType | string | None | NORMAL、I3S、3DTILES | 輸出檔案格式 |

In [ ]:
from OViewPy.server import Server
from OViewPy.varstruct import LAYER_TYPE
# 綁定Server物件
server = Server(url="http://127.0.0.1:8080")
# 設定layer名稱
layerName = "photogrammetry"
# 設定exportType(可輸入'NORMAL' or 'I3S' or '3DTILES')
exportType = "NORMAL"
# 轉檔傾斜攝影圖層，並會生成其對應之DB
id = server.convertPhotogrammetry(layerName=layerName,
                                  layerDBFile=layerName + ".DB",
                                  inputFileName="C:\\Users\\vaness.chen\\Desktop\\3DTEST\\基本圖資\\模型\\一門三節坊\\Acute3D\\一門三節坊\\Data\\LODTreeExport.xml", 
                                  terrainDBFile="C:\\Users\\vaness.chen\\Desktop\\3DTEST\\基本圖資\\高程\\Terrain20M.DB", 
                                  terrainName="terrain",
                                  exportType=exportType)
# 回傳轉檔進度直到轉檔完成
server.getConvert3dProgressToFinish(id)

# 將此圖層上架至伺服器
if exportType.upper() == "NORMAL":
    server.save3dToServer(dbLayerName=layerName, layerName=layerName, layerType=LAYER_TYPE.OV_PHOTOGRAMMETRYMODEL.name, dbUrl = layerName + ".DB")
elif exportType.upper() == "I3S":
    server.save3dToServer(dbLayerName=layerName, layerName=layerName, layerType=LAYER_TYPE.OV_OGCI3S.name, dbUrl = layerName + ".DB")

## 將管線圖層轉檔並上架至伺服器

透過綁定的server物件，並提供欲設定的圖層名稱(`layerName`)、檔案格式(`mlayerType`)及輸出格式(`exportType`)，將管線圖層轉檔並上架至伺服器  
由於3d的轉檔需要較久的時間，因此轉檔的函式會回傳一個id(如: `server.convertPipeline`)  
可以透過此id，去跟server詢問進度  
當轉檔成功透過`server.save3dToServer`依DB位置將圖層上架至伺服器(若輸出檔案格式為3DTILES則不上架)  
| 參數名稱 | Type | 預設值 | 可輸入值 | 說明 |
| :-----: | :---: | :---: | :---: | :--: |
| layerName | string | None | --- | 上架後的圖層名稱 |
| terrainName | string | None | --- | 初始地形圖層名稱 |
| terrainDBFile | string | None | --- | 初始地形圖層位置 |
| sourceFile | string | None | --- | 欲轉檔檔案位置 |
| epsg | int | None | --- | 座標參考系統 |
| mlayerType | string | None | PIPELINE、MANHOLE、EQUIPMENT | 欲轉檔檔案格式 |
| exportType | string | None | NORMAL、I3S、3DTILES | 輸出檔案格式 |
| layerDBFile | string | 預設名稱與layerName相同 | --- | 轉檔後產生的DB檔名稱 |








In [ ]:
from OViewPy.server import Server
from OViewPy.varstruct import LAYER_TYPE
# 綁定Server物件
server = Server(url="http://127.0.0.1:8080")
# 設定layer名稱
layerName = "EQUIPMENTTry"
# 設定mlayerType(可輸入'PIPELINE' or 'MANHOLE' or 'EQUIPMENT')
mlayerType = "EQUIPMENT"
# 設定exportType(可輸入'NORMAL' or 'I3S' or '3DTILES')
exportType = "NORMAL"
# 轉檔管線、人手孔及設備圖層，並會生成其對應之DB
id = server.convertPipeline(layerName=layerName,
                            terrainName="terrain",
                            terrainDBFile="C:\\Users\\vaness.chen\\Desktop\\3DTEST\\基本圖資\\高程\\Terrain20M.DB", 
                            sourceFile="C:\\Users\\vaness.chen\\Downloads\\台中全區SHP_T\\台中全區SHP_T1\\一般供電_設備.shp",
                            epsg="3826",
                            mlayerType=mlayerType,
                            exportType=exportType,
                            layerDBFile=layerName + ".DB",
                            )
# 回傳轉檔進度直到轉檔完成
server.getConvert3dProgressToFinish(id)
# 將此圖層上架至伺服器
if exportType.upper() == "NORMAL":
    server.save3dToServer(dbLayerName=layerName, layerName=layerName, layerType=LAYER_TYPE.OV_PIPELINE.name, dbUrl = layerName + ".DB")
elif exportType.upper() == "I3S":
    server.save3dToServer(dbLayerName=layerName, layerName=layerName, layerType=LAYER_TYPE.OV_OGCI3S.name, dbUrl = layerName + ".DB")

## 將點雲圖層轉檔並上架至伺服器

透過綁定的server物件，並提供欲設定的圖層名稱(`layerName`)及輸出格式(`exportType`)，將點雲圖層轉檔並上架至伺服器  
由於3d的轉檔需要較久的時間，因此轉檔的函式會回傳一個id(如: `server.convertPointcloud`)  
可以透過此id，去跟server詢問進度  
當轉檔成功透過`server.save3dToServer`依DB位置將圖層上架至伺服器(若輸出檔案格式為3DTILES則不上架)  
| 參數名稱 | Type | 預設值 | 可輸入值 | 說明 |
| :-----: | :---: | :---: | :---: | :--: |
| layerName | string | None | --- | 上架後的圖層名稱 |
| layerDBFile | string | 預設名稱與layerName相同 | --- | 轉檔後產生的DB檔名稱 |
| pointcloudFileNames | string | None | --- | 欲轉檔檔案位置 |
| terrainName | string | None | --- | 初始地形圖層名稱 |
| terrainDBFile | string | None | --- | 初始地形圖層位置 |
| sourceEPSG | int | None | --- | 座標參考系統 |
| exportType | string | None | NORMAL、I3S、3DTILES | 輸出檔案格式 |


In [ ]:
from OViewPy.server import Server
from OViewPy.varstruct import LAYER_TYPE
# 綁定Server物件
server = Server(url="http://127.0.0.1:8080")
# 設定layer名稱
layerName = "PointcloudTry"
# 設定exportType(可輸入'NORMAL' or 'I3S' or '3DTILES')
exportType = "NORMAL"
# 轉檔管線圖層，並會生成其對應之DB
id = server.convertPointcloud(layerName=layerName,
                              layerDBFile=layerName + ".DB",
                              pointcloudFileNames="C:\\Users\\vaness.chen\\Downloads\\點雲圖層\\點雲圖層\\field-1.las",
                              terrainName="terrain",
                              terrainDBFile="C:\\Users\\vaness.chen\\Desktop\\3DTEST\\基本圖資\\高程\\Terrain20M.DB",
                              sourceEPSG="3826",
                              exportType=exportType
                             )
# 回傳轉檔進度直到轉檔完成
server.getConvert3dProgressToFinish(id)
# 將此圖層上架至伺服器
if exportType.upper() == "NORMAL":
    server.save3dToServer(dbLayerName=layerName, layerName=layerName, layerType=LAYER_TYPE.OV_POINTCLOUD.name, dbUrl = layerName + ".DB")
elif exportType.upper() == "I3S":
    server.save3dToServer(dbLayerName=layerName, layerName=layerName, layerType=LAYER_TYPE.OV_OGCI3S.name, dbUrl = layerName + ".DB")

## 將shp檔轉成ModelSet並上架至伺服器

透過綁定的server物件，並提供欲設定的圖層名稱(`layerName`)及輸出格式(`exportType`)，將shp檔轉檔模型集並上架至伺服器  
由於3d的轉檔需要較久的時間，因此轉檔的函式會回傳一個id(如: `server.shpToModelSet`)  
可以透過此id，去跟server詢問進度  
當轉檔成功透過`server.save3dToServer`依DB位置將圖層上架至伺服器(若輸出檔案格式為3DTILES則不上架)  
| 參數名稱 | Type | 預設值 | 可輸入值 | 說明 |
| :-----: | :---: | :---: | :---: | :--: |
| layerName | string | None | --- | 上架後的圖層名稱 |
| layerDBFile | string | 預設名稱與layerName相同 | --- | 轉檔後產生的DB檔名稱 |
| terrainName | string | None | --- | 初始地形圖層名稱 |
| terrainDBFile | string | None | --- | 初始地形圖層位置 |
| sourceFileName | string | None | --- | 欲轉檔檔案位置 |
| heightField | string | None | --- | 屋頂高欄位的名稱 |
| epsg | int | None | --- | 座標參考系統 |
| exportType | string | None | NORMAL、I3S、3DTILES | 輸出檔案格式 |


In [ ]:
from OViewPy.server import Server
from OViewPy.varstruct import LAYER_TYPE
# 綁定Server物件
server = Server(url="http://127.0.0.1:8080")
# 設定layer名稱
layerName = "shpModelSetTry"
# 設定exportType(可輸入'NORMAL' or 'I3S' or '3DTILES')
exportType = "NORMAL"
# 轉檔管線圖層，並會生成其對應之DB
id = server.shpToModelSet(layerName=layerName,
                          layerDBFile=layerName + ".DB",
                          terrainName="terrain",
                          terrainDBFile="C:\\Users\\vaness.chen\\Desktop\\3DTEST\\基本圖資\\高程\\Terrain20M.DB",
                          sourceFileName="C:\\Users\\vaness.chen\\Desktop\\3DTEST\\基本圖資\\模型\\modelset練習用\\TaipeiBuilding2.shp",
                          heightField="屋頂高程",
                          epsg=4326,
                          exportType=exportType)
# 回傳轉檔進度直到轉檔完成
server.getConvert3dProgressToFinish(id)
# 將此圖層上架至伺服器
if exportType.upper() == "NORMAL":
    server.save3dToServer(dbLayerName=layerName, layerName=layerName, layerType=LAYER_TYPE.OV_MODELSET.name, dbUrl = layerName + ".DB")
elif exportType.upper() == "I3S":
    server.save3dToServer(dbLayerName=layerName, layerName=layerName, layerType=LAYER_TYPE.OV_OGCI3S.name, dbUrl = layerName + ".DB")

## 將huge檔轉成Model並上架至伺服器

透過綁定的server物件，並提供欲設定的圖層名稱(`layerName`)及輸出格式(`exportType`)，將huge檔轉檔模型並上架至伺服器  
由於3d的轉檔需要較久的時間，因此轉檔的函式會回傳一個id(如: `server.hugeModelToModelLayer`)  
可以透過此id，去跟server詢問進度  
當轉檔成功透過`server.save3dToServer`依DB位置將圖層上架至伺服器(若輸出檔案格式為3DTILES則不上架)  
| 參數名稱 | Type | 預設值 | 可輸入值 | 說明 |
| :-----: | :---: | :---: | :---: | :--: |
| dbPath | string | 預設名稱與layerName相同 | --- | 轉檔後產生的DB檔名稱 |
| layerName | string | None | --- | 上架後的圖層名稱 |
| terrainDBFile | string | None | --- | 初始地形圖層位置 |
| terrainName | string | None | --- | 初始地形圖層名稱 |
| sourceFileName | string | None | --- | 欲轉檔檔案位置 |
| exportType | string | None | NORMAL、I3S、3DTILES | 輸出檔案格式 |

In [ ]:
from OViewPy.server import Server
from OViewPy.varstruct import LAYER_TYPE

# 綁定Server物件
server = Server(url="http://127.0.0.1:8080")
# 設定layer名稱
layerName = "hugeModelTry"
# 設定exportType(可輸入'NORMAL' or 'I3S' or '3DTILES')
exportType = "NORMAL"
# 轉檔HugeModel圖層，並會生成其對應之DB
id = server.hugeModelToModelLayer(dbPath=layerName + ".DB",
                                  layerName=layerName,
                                  terrainDBFile="C:\\Users\\vaness.chen\\Desktop\\3DTEST\\基本圖資\\高程\\Terrain20M.DB",
                                  terrainName="terrain",
                                  sourceFileName="C:\\Users\\vaness.chen\\Desktop\\3DTEST\\基本圖資\\模型\\HugeModel練習用\\-Z_Y.obj",
                                  exportType=exportType
                                 )
# 回傳轉檔進度直到轉檔完成
server.getConvert3dProgressToFinish(id)
# 將此圖層上架至伺服器
if exportType.upper() == "NORMAL":
    server.save3dToServer(dbLayerName=layerName, layerName=layerName, layerType=LAYER_TYPE.OV_MODEL.name, dbUrl = layerName + ".DB")
elif exportType.upper() == "I3S":
    server.save3dToServer(dbLayerName=layerName, layerName=layerName, layerType=LAYER_TYPE.OV_OGCI3S.name, dbUrl = layerName + ".DB")